In [1]:
import pandas as pd 
import numpy as np
import pathlib 
import os 
import warnings
warnings.filterwarnings("ignore")

xls = pd.ExcelFile('./All_panel_designs_20250601.xlsx')
df = dict()
for sheet_name in [item for item in xls.sheet_names if item != "Runs_tracking"]:
    print(f"Processing {sheet_name}...")
    df[sheet_name] = pd.read_excel("./All_panel_designs_20250601.xlsx", sheet_name=sheet_name)    
    beddf = df[sheet_name][["Amplicon", "hg19"]]
    beddf["hg19"] = beddf["hg19"].apply(lambda x: x.replace(" ", ""))  # Remove any extra spaces
    beddf["chrom"] = beddf["hg19"].apply(lambda x: x.split(":")[0]) 
    beddf["start"] = beddf["hg19"].apply(lambda x: int(x.split(":")[1].split("-")[0]))
    beddf["end"] = beddf["hg19"].apply(lambda x: int(x.split(":")[1].split("-")[1]))
    beddf[["chrom", "start", "end", "Amplicon"]].\
        sort_values(by = "Amplicon").\
            to_csv(
                f"./beds/{sheet_name}.bed", 
                sep="\t", 
                header=False, 
                index=False
            )

Processing Lung_panel_version_1.1...
Processing CRC_panel_version_3.1...
Processing CRC_panel_version_1.1...
Processing CRC_panel_version_1.2...
Processing CRC_panel_version_2.1...
Processing Truong_amplicon_PBMC_v1.1...
